In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker, cm, colors
import dask
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

# import crh_percentiles as crhp

In [ ]:
# functions
def q2rh(q,p,t, **kwargs):
    """
    Returns xarray of relative humidity
    rh = q/qs*100
    qs = 0.622*es/P
    es = 610.94 exp(17.625*T/(T+243.04)) for T >= 0C
       = 611.21 exp(22.587*T/(T+273.86)) for T <  0C
    """
    t = t - 273.15 # convert to deg C
    # units of es are Pa
    es_warm = 610.94 * np.exp(17.625*t.where(t>=0, other=0)/(t.where(t>=0, other=0)+243.04))
    es_cold = 611.21 * np.exp(22.587*t.where(t<0, other=0)/(t.where(t<0, other=0)+273.86))
    es = es_warm + es_cold
    qs = 0.622*es/p
    rh = q/qs * 100
    return rh

def crh_percentiles(rh, lev="lev", return_crh=False):
    """
    Integreate column realitive humidity then bin by percentiles.
    Bins are 0-100 by 1s. Returns bin edges and crh_percentiles as
    mid_bin values. 
    
    Returns: bins, crh_percs
        if return_crh is True, returns (crh, (bins, crh_percs))
    """
    crh = rh.integrate(lev)
    bins = np.arange(0,101)
    crh_percs = np.zeros(crh.shape)
    
    for i in range(len(bins)-1):
        perc_thres_lower = np.nanpercentile(crh, bins[i])
        perc_thres_upper = np.nanpercentile(crh, bins[i+1])
        crh_percs = np.where((crh>=perc_thres_lower)&(crh<perc_thres_upper), 
                             (bins[i]+bins[i+1])/2, crh_percs)
    crh_percs = xr.DataArray(crh_percs, dims=crh.dims, coords=crh.coords, 
                             attrs={"long_name":"column relative humidity binned percentiles","units":"%"})
    if return_crh:
        return crh, (bins, crh_percs)
    else:
        return bins, crh_percs
    return bins, crh_percs

def stream_function(omega, crh_perc, bins):
    """
    Calculate the mean profiles for given omega (Pa/s)
    for each CRH percentile bin.
    
    Input:  vertical velocity (omega, unit: Pa/s),  
            column relative humidity percentiles (crh_perc, unit: %),
            bin edges (bins, unit: %) same as from crh_percentiles.
            
    Output: stream function (phi_rp) in coordinates of CRH and pressure.
                should be of shape (100, nlevs)
            if return_omega is True, returns (w_rp, phi_rp) as tuple
            
    """
    w_rp   = np.zeros((len(bins)-1, omega.shape[1])) # shape of crh mid_bins and pres levels
    phi_rp = np.zeros((len(bins)-1, omega.shape[1])) # shape of crh mid_bins and pres levels
    for i in range(20, len(bins)-1):
        w_rp[i,:] = omega.where((crh_perc>= bins[i])&(crh_perc<bins[i+1])).mean(skipna=True, dim=["time","ncol"])
        # w_rp[i,:] = np.nanmean(np.where((crhp>=bins[i])&(crhp<=bins[i+1]), 
        #                             omega, np.nan), axis=(0,2))
        if (i==20):
            phi_rp[i,:] = 0.01/9.8*w_rp[i,:]
        else:
            phi_rp[i,:] = phi_rp[i-1]+ 0.01/9.8*w_rp[i,:] 
        # print(i, 30, ":", w_rp[i,30], phi_rp[i,30])
    return phi_rp

def calc_rho(qv, t, p):
    Tv = (1 + 0.61*qv)*t
    rho = p / (287*Tv)
    return rho

def w2omega(w, p, t, qv):
    rho = calc_rho(qv, t, p)
    g=9.8 #m/s2
    omega = -rho*g*w # (kg/m/s2)/s = Pa/s
    return omega

def omega2w(omega, p, t, qv):
    rho = calc_rho(qt, t, p)
    g=9.8
    w = -omega/(rho*g) # m/s
    return w

In [4]:
model="SCREAM"
chunks={"time":8, "ncol":1024}
###########################
file_dir = "/work/bb1153/b380883/TWP/"
if model=="SCREAM":
    w_file = file_dir + "TWP_3D_SCREAM_wap_20200130-20200228.nc"
    q_file = file_dir + "TWP_3D_SCREAM_cltotal_20200130-20200228.nc"
    p_file = file_dir + "TWP_3D_SCREAM_pa_20200130-20200228.nc"
    t_file = file_dir + "TWP_3D_SCREAM_ta_20200130-20200228.nc"
else:
    w_file = file_dir + "TWP_3D_{}_wa_20200130-20200228.nc".format(model)
    q_file = file_dir + "TWP_3D_{}_cltotal_20200130-20200228.nc".format(model)
    t_file = file_dir + "TWP_3D_{}_ta_20200130-20200228.nc".format(model)
    p_file = file_dir + "TWP_3D_{}_pa_20200130-20200228.nc".format(model)
q = xr.open_dataset(q_file, chunks=chunks).cltotal.isel(time=slice(-80,-1))
t = xr.open_dataset(t_file, chunks=chunks).ta.isel(time=slice(-80,-1))
if model=="SCREAM":
    omega = xr.open_dataset(w_file, chunks=chunks).wap.isel(time=slice(-80,-1))
    p = omega.lev
else:
    p = xr.open_dataset(p_file, chunks=chunks).pa.isel(time=slice(-80,-1))
    w = xr.open_dataset(w_file, chunks=chunks).wa.isel(time=slice(-80,-1))
    omega = w2omega(w, p, t, q)
# print(omega.shape, q.shape, t.shape, p.shape)
omega

<xarray.DataArray 'wap' (time: 79, lev: 128, ncol: 61250)>
dask.array<getitem, shape=(79, 128, 61250), dtype=float32, chunksize=(8, 128, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2020-02-19 00:00:00 ... 2020-02-28 18:00:00
    lon      (ncol) float64 dask.array<chunksize=(1024,), meta=np.ndarray>
    lat      (ncol) float64 dask.array<chunksize=(1024,), meta=np.ndarray>
  * lev      (lev) float64 2.58 3.23 3.88 4.597 ... 987.2 991.0 994.9 998.5
Dimensions without coordinates: ncol
Attributes:
    standard_name:  lagrangian_tendency_of_air_pressure
    long_name:      Vertical velocity (pressure)
    units:          Pa s-1
    CDI_grid_type:  unstructured
    mdims:          1
    origin:         OMEGA

In [5]:
rh = q2rh(q, p, t)
print(rh.shape)#, bins.shape, crh_percs.shape, phi_rp.shape)

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
(79, 128, 61250)


In [ ]:
bins, crh_percs = crh_percentiles(rh)
print(bins.shape, crh_percs.shape)

[########################################] | 100% Completed | 26.4s
[########################################] | 100% Completed |  8.1s
[########################################] | 100% Completed |  7.8s
[########################################] | 100% Completed |  7.8s
[########################################] | 100% Completed |  7.8s
[########################################] | 100% Completed |  7.4s
[########################################] | 100% Completed |  7.8s
[########################################] | 100% Completed |  7.5s
[########################################] | 100% Completed |  7.7s
[########################################] | 100% Completed |  7.5s
[########################################] | 100% Completed |  8.0s
[########################################] | 100% Completed |  7.6s
[########################################] | 100% Completed |  7.4s
[########################################] | 100% Completed |  7.8s
[########################################] | 100

In [ ]:
phi_rp = stream_function(w, crh_percs, bins)
print(phi_rp.shape)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,12))
bin_mid = (bins[:-1]+bins[1:])/2
pc = ax.contourf(bin_mid, w.lev.values, phi_rp.T)
ax.set_ylim([1000,0])
plt.colorbar(pc, ax=ax )
plt.savefig("../plots/{}_streamfunction_twp_2.png".format(model))
plt.show()